# IMPORT LIBRARIES


In [6]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error as sklearn_mean_absolute_error
from sklearn.metrics import mean_squared_error as sklearn_mean_squared_error
from sklearn.metrics import r2_score
from scipy.sparse import random
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

# IMPORT DATASET

In [2]:
df = pd.read_csv('Albrecht.csv')
df.head(5)

,Input,Output,Inquiry,File,FPAdj,RawFPcounts,AdjFP,Effort
0,25,150,75,60,1.00,1750.00,1750,102.4
1,193,98,70,36,1.00,1902.00,1902,105.2
2,70,27,0,12,0.80,535.00,428,11.1
3,40,60,20,12,1.15,660.00,759,21.1
4,10,69,1,9,0.90,478.89,431,28.8


# PREPROCESSING

In [3]:
X=df[['Output',	'Inquiry',	'RawFPcounts',	'AdjFP']]
y=df[['Effort']]


#DATA SPLITTING

In [8]:


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


# FEATURE SCALING

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#Stacked PCR

In [10]:
from sklearn.decomposition import PCA


# Apply PCA for dimensionality reduction
n_components = min(X_train_scaled.shape)  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Define base regressors
base_regressors = [
    ('linear_regression', LinearRegression()),
    ('svr', SVR(kernel='linear')),
    ('random_forest', RandomForestRegressor(n_estimators=50, random_state=0))
]

def mean_absolute_error(y_true, y_pred):
    mae = sklearn_mean_absolute_error(y_true, y_pred)
    maximum_value = 100
    minimum_value = 0
    converted_value = mae / (maximum_value - minimum_value)
    return converted_value

def mean_squared_error(y_true, y_pred):
    mse = sklearn_mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    maximum_value = 100
    minimum_value = 0
    converted_value = rmse / (maximum_value - minimum_value)
    return converted_value

# def mean_absolute_percentage_error(y_true, y_pred):
#     mmre = sklearn_mean_absolute_percentage_error(y_true, y_pred)
#     maximum_value = 10
#     minimum_value = 0
#     converted_value = mmre / (maximum_value - minimum_value)
#     return converted_value

# Define the meta-regressor
meta_regressor = LinearRegression()

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_pca, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_pca)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.02742945748146245
Root Mean Squared Error (RMSE): 0.035558482670355114
R-squared (R2) Score: -0.06884906490324627
Mean Absolute Percentage Error (MMRE): 1.0781486632401456


#Linear Regression
and hyper parameter tunuing

In [ ]:
regressor = LinearRegression()

# Hyperparameter Tuning
param_grid = {
    'fit_intercept': [True, False],
    'positive': [True, False],  # Include positive parameter
    'n_jobs': [-1, 1],           # Include n_jobs parameter
    'copy_X': [True, False],      # Include copy_X parameter

}

grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Best parameters found
print("Best Parameters:", grid_search.best_params_)

# Evaluating the model with best parameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)




Best Parameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'positive': True}
Mean Absolute Error (MAE): 0.13207582387798605
Root Mean Squared Error (RMSE): 0.14718686160549158
R-squared (R2) Score: -0.3120964637594743
Mean Absolute Percentage Error (MMRE): 2.282969112711116


#Support Vector Machine

In [11]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)
support=SVR(kernel='rbf')
support.fit(X_train,y_train)
y_pred=support.predict(X_test)

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.0897194952310632
Root Mean Squared Error (RMSE): 0.11586498271521764
R-squared (R2) Score: 0.18692224924549028
Mean Absolute Percentage Error (MMRE): 0.8894768700262216


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#Random Forest Regression

In [ ]:
# Choose a specific value of n_estimators
n_estimators_value = 500

# Create and fit the RandomForestRegressor with the chosen value of n_estimators
random = RandomForestRegressor(n_estimators=n_estimators_value, max_features=5, random_state=42)
random.fit(X_train, y_train)

# Predict on the test set
y_pred = random.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics for the chosen value of n_estimators

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)


<ipython-input-22-e7d0fefdf2e5>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)


Mean Absolute Error (MAE): 0.03248560000000007
Root Mean Squared Error (RMSE): 0.046096773179909636
R-squared (R2) Score: 0.8713029104546292
Mean Absolute Percentage Error (MMRE): 0.6090649524382559


#KNeighbor


In [ ]:

# Splitting the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)

# Instantiate and fit the KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3, weights='uniform')
neigh.fit(X_train, y_train)

# Predict on the test data
y_pred = neigh.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.06360000000000002
Root Mean Squared Error (RMSE): 0.08010187957289837
R-squared (R2) Score: 0.6113907906559575
Mean Absolute Percentage Error (MMRE): 1.0051247709408344


#Bagging

In [ ]:
from sklearn.ensemble import BaggingRegressor

# Replace DecisionTreeRegressor with BaggingRegressor
bagging_dtree = BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=22),
                                 n_estimators=10,  # You can adjust the number of base estimators (trees)
                                 random_state=22)
bagging_dtree.fit(X_train, y_train)
predict = bagging_dtree.predict(X_test)
# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.06360000000000002
Root Mean Squared Error (RMSE): 0.08010187957289837
R-squared (R2) Score: 0.6113907906559575
Mean Absolute Percentage Error (MMRE): 1.0051247709408344


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


#Boosting

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
gradient_booster = GradientBoostingRegressor(learning_rate=0.1)
gradient_booster.fit(X_train,y_train)
y_pred = gradient_booster.predict(X_test)


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.054924366810221156
Root Mean Squared Error (RMSE): 0.07399786695915815
R-squared (R2) Score: 0.6683606335122083
Mean Absolute Percentage Error (MMRE): 0.7264603606374408


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#CATBOOST

In [ ]:

# Instantiate the CatBoostRegressor model
catboost_model = CatBoostRegressor(iterations=100, depth=8, learning_rate=0.1, loss_function='MAE', random_seed=0)

# Fit the model to the training data
catboost_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = catboost_model.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

0:	learn: 12.8405252	total: 46.9ms	remaining: 4.64s
1:	learn: 12.4904727	total: 47.1ms	remaining: 2.31s
2:	learn: 11.8780043	total: 47.3ms	remaining: 1.53s
3:	learn: 10.9878248	total: 47.5ms	remaining: 1.14s
4:	learn: 9.9576217	total: 48ms	remaining: 912ms
5:	learn: 9.1163001	total: 48.5ms	remaining: 760ms
6:	learn: 8.5699003	total: 48.9ms	remaining: 649ms
7:	learn: 8.3073849	total: 49.2ms	remaining: 566ms
8:	learn: 8.0908686	total: 49.6ms	remaining: 502ms
9:	learn: 7.8473584	total: 50ms	remaining: 450ms
10:	learn: 7.2215341	total: 50.4ms	remaining: 408ms
11:	learn: 6.7019155	total: 50.8ms	remaining: 373ms
12:	learn: 6.1899810	total: 51.4ms	remaining: 344ms
13:	learn: 5.7710066	total: 51.8ms	remaining: 318ms
14:	learn: 5.4101600	total: 52.4ms	remaining: 297ms
15:	learn: 5.0706425	total: 53ms	remaining: 278ms
16:	learn: 4.8042885	total: 53.4ms	remaining: 261ms
17:	learn: 4.3420257	total: 53.9ms	remaining: 246ms
18:	learn: 4.0267311	total: 54.6ms	remaining: 233ms
19:	learn: 3.8535917	tot

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100)
predict = gradient_booster.predict(X_test)

NameError: name 'gradient_booster' is not defined

#Stacking using decision tree


In [ ]:

# Define base models
dt1 = DecisionTreeRegressor(max_depth=3, random_state=42)
dt2 = DecisionTreeRegressor(max_depth=5, random_state=42)

# Fit the base models on the training data
dt1.fit(X_train, y_train)
dt2.fit(X_train, y_train)

# Make predictions on the test data using the base models
dt1_y_pred = dt1.predict(X_test)
dt2_y_pred = dt2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((dt1.predict(X_train), dt2.predict(X_train)))
stacking_test = np.column_stack((dt1_y_pred, dt2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.07598906507936506
Root Mean Squared Error (RMSE): 0.10351836410401546
R-squared (R2) Score: 0.3509734317827967
Mean Absolute Percentage Error (MMRE): 0.7594789637628556


<ipython-input-27-6d523f838c4a>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


# **Stacking using SVM**

In [12]:
# Define base models
svm1 = SVR(kernel='linear')
svm2 = SVR(kernel='rbf')

# Fit the base models on the training data
svm1.fit(X_train, y_train)
svm2.fit(X_train, y_train)

# Make predictions on the test data using the base models
svm1_y_pred = svm1.predict(X_test)
svm2_y_pred = svm2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((svm1.predict(X_train), svm2.predict(X_train)))
stacking_test = np.column_stack((svm1_y_pred, svm2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.08391599999999988
Root Mean Squared Error (RMSE): 0.10373607164337754
R-squared (R2) Score: 0.34824064984713443
Mean Absolute Percentage Error (MMRE): 0.9136291665086478


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-12-98cbb8369f44>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


# **Stacking using Random Forest**

In [ ]:
# Define base models
rf1 = RandomForestRegressor(n_estimators=10, random_state=42)
rf2 = RandomForestRegressor(n_estimators=50, random_state=42)

# Fit the base models on the training data
rf1.fit(X_train, y_train)
rf2.fit(X_train, y_train)

# Make predictions on the test data using the base models
rf1_y_pred = rf1.predict(X_test)
rf2_y_pred = rf2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((rf1.predict(X_train), rf2.predict(X_train)))
stacking_test = np.column_stack((rf1_y_pred, rf2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

<ipython-input-50-b612a1d0897b>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf1.fit(X_train, y_train)
<ipython-input-50-b612a1d0897b>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf2.fit(X_train, y_train)
<ipython-input-50-b612a1d0897b>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


Mean Absolute Error (MAE): 0.05229999999999997
Root Mean Squared Error (RMSE): 0.07228776161979285
R-squared (R2) Score: 0.6835120138380811
Mean Absolute Percentage Error (MMRE): 0.6858677955547134


# Stacked XGBoost

In [ ]:
# Create the base models (XGBoost)
xgb_model1 = XGBRegressor(n_estimators=100, random_state=42)
xgb_model2 = XGBRegressor(n_estimators=100, random_state=42)


# Train the base models
xgb_model1.fit(X_train, y_train)
xgb_model2.fit(X_train, y_train)

# Make predictions with the base models
xgb_predictions1 = xgb_model1.predict(X_test)
xgb_predictions2 = xgb_model2.predict(X_test)

# Create a meta-model (XGBoost) to combine predictions
meta_model = XGBRegressor(n_estimators=100, random_state=42)


# Combine the predictions from base models as meta-features
meta_features = pd.DataFrame({'XGB1': xgb_predictions1, 'XGB2': xgb_predictions2})

# Train the meta-model on the predictions
meta_model.fit(meta_features, y_test)

# Make predictions with the meta-model
meta_predictions = meta_model.predict(meta_features)

# Calculate the MAE for the ensemble model

mae = mean_absolute_error(y_test, meta_predictions)
mse = mean_squared_error(y_test, meta_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, meta_predictions)

print(f"Ensemble MAE: {mae}")

print(f"Ensemble RMSE: {rmse}")
print(f"Ensemble R-squared: {r2}")

mmre = mean_absolute_percentage_error(y_test, meta_predictions)
print("Mean Absolute Percentage Error (MMRE):", mmre)






Ensemble MAE: 8.865356445327599e-06
Ensemble RMSE: 0.0031485280312197926
Ensemble R-squared: 0.9999999992262097
Mean Absolute Percentage Error (MMRE): 0.00010559165485515503


# Stacked GradientBoost

In [ ]:
# Define base models
base_model1 = GradientBoostingRegressor(n_estimators=100, random_state=42)
base_model2 = GradientBoostingRegressor(n_estimators=200, random_state=42)

# Define the stacked model
stacked_model = StackingRegressor(
    estimators=[
        ('model1', base_model1),
        ('model2', base_model2)
    ],
    final_estimator=GradientBoostingRegressor(n_estimators=50, random_state=42)
)

# Fit the stacked model
stacked_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = stacked_model.predict(X_test)


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.10447766643292215
Root Mean Squared Error (RMSE): 0.1383296656017097
R-squared (R2) Score: -0.15893299877662037
Mean Absolute Percentage Error (MMRE): 0.9306135787380804


#Stacked nnet

In [ ]:

# Define base regressors (neural networks)
base_regressors = [
    ('mlp_1', MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)),
    ('mlp_2', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=0)),
    ('mlp_3', MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=1000, random_state=0))
]

# Define the meta-regressor
meta_regressor = MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklear

Mean Absolute Error (MAE): 0.17950794019158983
Root Mean Squared Error (RMSE): 0.21839612332315386
R-squared (R2) Score: -1.8888003291499844
Mean Absolute Percentage Error (MMRE): 1.5746805064298586


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


#Stacked elasticnet


In [ ]:
# Define base regressors (GLMNet - ElasticNet)
base_regressors = [
    ('elastic_net_1', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)),
    ('elastic_net_2', ElasticNet(alpha=0.01, l1_ratio=0.7, random_state=0)),
    ('elastic_net_3', ElasticNet(alpha=0.001, l1_ratio=0.9, random_state=0))
]

# Define the meta-regressor
meta_regressor = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.12205651673984103
Root Mean Squared Error (RMSE): 0.14090559417676862
R-squared (R2) Score: -0.20249739992757565
Mean Absolute Percentage Error (MMRE): 2.241974590418334


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.501e+03, tolerance: 1.751e+00
  model = cd_fast.enet_coordinate_descent(


#Averaging

In [ ]:


# Define base models
models = [
    RandomForestRegressor(random_state=42),
    LinearRegression(),
    DecisionTreeRegressor(random_state=42),  # Include Decision Tree
    KNeighborsRegressor(),                  # Include k-Nearest Neighbors
    GradientBoostingRegressor(random_state=42)  # Include Gradient Boosting
]

# Train base models and make predictions
predictions = []
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions.append(y_pred)

# Average predictions
avg_predictions = sum(predictions) / len(predictions)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, avg_predictions)
rmse = r_mse(y_test, avg_predictions) # RMSE
r2 = r_squared(y_test, avg_predictions)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

<ipython-input-38-ec5ae82c07e8>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: y_true and y_pred have different number of output (1!=5)